# Model Training

In this notebook, we will ask you a series of questions regarding model selection. Based on your responses, we will ask you to create the ML models that you've chosen. 

The bonus step is completely optional, but if you provide a sufficient third machine learning model in this project, we will add `1000` points to your Kahoot leaderboard score.

**Note**: Use the dataset that you've created in your previous data transformation step (not the original model).

## Questions
Is this a classification or regression task?  

Classification

Are you predicting for multiple classes or binary classes?  

Binary

Given these observations, which 2 (or possibly 3) machine learning models will you choose?  

Random Forest (for interpretability + robustness), Logistic Regression (baseline), and XGBoost (high performance with imbalance handling).

## First Model

Using the first model that you've chosen, implement the following steps.

### 1) Create a train-test split

Use your cleaned and transformed dataset to divide your features and labels into training and testing sets. Make sure you’re only using numeric or properly encoded features.  

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder


In [2]:
# Load and prepare transformed data
transactions = pd.read_csv("../data/bank_transactions.csv")
transactions = transactions.drop(columns=['nameOrig', 'nameDest', 'isFlaggedFraud'])
transactions['type'] = LabelEncoder().fit_transform(transactions['type'])
transactions['errorOrig'] = transactions['oldbalanceOrg'] - transactions['newbalanceOrig'] - transactions['amount']
transactions['errorDest'] = transactions['newbalanceDest'] - transactions['oldbalanceDest'] - transactions['amount']


In [3]:
# Features & Target
X = transactions.drop(columns=['isFraud'])
y = transactions['isFraud']

In [4]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [5]:
# Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

### 2) Search for best hyperparameters
Use tools like GridSearchCV, RandomizedSearchCV, or model-specific tuning functions to find the best hyperparameters for your first model.

In [ ]:
# Hyperparameter tuning
param_dist = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5],
}

rfc = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)

search_rfc = RandomizedSearchCV(
    rfc, 
    param_distributions=param_dist, 
    n_iter=5, 
    scoring='recall', 
    cv=3, 
    random_state=42,
    verbose=1,
    n_jobs=-1
)

search_rfc.fit(X_resampled, y_resampled)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


### 3) Train your model
Select the model with best hyperparameters and generate predictions on your test set. Evaluate your models accuracy, precision, recall, and sensitivity.  

In [ ]:
# Best model
from sklearn.metrics import classification_report, confusion_matrix


print(confusion_matrix(y_test, y_pred_rfc))
print(classification_report(y_test, y_pred_rfc, digits=4))


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

## Second Model

Create a second machine learning object and rerun steps (2) & (3) on this model. Compare accuracy metrics between these two models. Which handles the class imbalance more effectively?

Create as many code-blocks as needed.

In [ ]:
# Step 2: Create and fit model
lr = LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')
lr.fit(X_resampled, y_resampled)

In [ ]:
# Step 3: Train model
y_pred_lr = lr.predict(X_test)

print("\n Logistic Regression Results:")
print(confusion_matrix(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr, digits=4))

### (Bonus/Optional) Third Model

Create a third machine learning model and rerun steps (2) & (3) on this model. Which model has the best predictive capabilities? 

Create as many code-blocks as needed.

In [ ]:
# Create and fit model
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=len(y_resampled[y_resampled == 0]) / len(y_resampled[y_resampled == 1]),
    random_state=42
)

xgb.fit(X_resampled, y_resampled)

In [ ]:
# Step 3: Train model
y_pred_xgb = xgb.predict(X_test)

print("\n XGBoost Results:")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=4))

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def get_metrics(y_true, y_pred, model_name):
    return {
        "Model": model_name,
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1 Score": f1_score(y_true, y_pred)
    }

results = [
    get_metrics(y_test, y_pred_rfc, "Random Forest"),
    get_metrics(y_test, y_pred_lr, "Logistic Regression"),
    get_metrics(y_test, y_pred_xgb, "XGBoost")
]

metrics_df = pd.DataFrame(results)
print(metrics_df)